In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from playsound import playsound # 외부 스피커 재생
from pathlib import Path
import pygame
import time

# client.audio.transcriptions.create(
#     file=..., # 필수: 오디오 파일 객체
#     model="whisper-1", # 필수: 모델 이름 (현재는 whisper-1 고정)
#     prompt=..., # 선택: 힌트 문장 제공 (맥락 강화)
#     response_format=..., # 선택: 응답 형식 ("json", "text", "srt", "verbose_json", "vtt")
#     temperature=..., # 선택: 창의성/랜덤성 조절 (0.0~1.0)
#     language=..., # 선택: 입력 음성의 언어 (예: "en", "ko", "es")
#     timestamp_granularities=... # 선택: 세그먼트 정보 단위 설정 (베타)
# )

# 환경 변수 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=api_key)

audio_file = open("speech.mp3", "rb")
response = client.audio.transcriptions.create(
 file=audio_file,
 model="whisper-1",
 response_format="text"
)
print(response)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
안녕! 오늘은 정말 화창한 봄날이네요. 조금 더워요.



In [3]:
audio_file = open("speech.mp3","rb")
transcription_json = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="verbose_json"
)

first_segment = transcription_json.segments[0]
print(f"Segment 1 start: {first_segment.start}, end: {first_segment.end},text:{first_segment.text}")

Segment 1 start: 0.0, end: 3.940000057220459,text: 안녕! 오늘은 정말 화창한 봄날이네요.


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
fs = 16000 # 샘플레이트 16kHz
seconds = 5 # 녹음 길이 (초)
print("지금부터 5초간 녹음합니다...")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait() # 녹음 완료 대기
write("live_input.wav", fs, recording) # WAV 파일로 저장
# 저장한 녹음 파일을 Whisper API로 전송
with open("live_input.wav", "rb") as audio_file:
 live_transcript = client.audio.transcriptions.create(
 file=audio_file,
 model="whisper-1",
 response_format="text",
 language="ko" # 예: 한국어로 가정하고 명시 (자동감지도 가능)
 )
print("실시간 녹음 변환 결과:", live_transcript)

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
from openai import OpenAI
from pathlib import Path
import time
# OpenAI API 클라이언트
client = OpenAI()
# 기본 설정
fs = 16000 # 샘플레이트 (16kHz)
seconds = 5 # 녹음 시간 (초)
filename = "live_input.wav"
# 1. 녹음 시작
print(" 지금부터 5초간 녹음합니다...")
recording = sd.rec(int(seconds * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()
write(filename, fs, recording)
print(" 녹음 완료!")
# 2. Whisper로 한국어 텍스트 변환
with open(filename, "rb") as audio_file:
    transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    response_format="text",
    language="ko"
 )
print("인식된 텍스트:", transcript)
print("\n [인식된 자막 - 한국어]")
print(transcript)
# 3. GPT로 영어 번역
messages = [
    {"role": "system", "content": "Translate the following Korean sentence into natural English."},
    {"role": "user", "content": transcript}
]
translation_response = client.chat.completions.create(
    model="gpt-4o", # 또는 "gpt-3.5-turbo"
    messages=messages,
    temperature=0.3
)
english_translation = translation_response.choices[0].message.content
print("\n [자동 번역 - 영어]")
print(english_translation)